## Preprocessing for ENSANUT 2012 - Componente de SALUD 

https://ensanut.insp.mx/encuestas/ensanut2012/descargas.php

- Información sobre el hogar
- Información sobre los integrantes del hogar
- Relación entre folio de hogar y folio de vivenda
- Cuestionario individual: Menores de 0 a 9 años de edad

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os


## Limpieza folio_vivienda


In [2]:
# read csv
folio_vivienda= pd.read_csv("../data/raw_salud/folio_vivienda.csv")

In [3]:
folio_vivienda.columns

Index(['folio_c', 'folio_v'], dtype='object')

Vars: 
- folio_c = folio consecutivo
- folio_v = folio de vivienda

In [4]:
folio_vivienda.isna().sum()

folio_c    0
folio_v    0
dtype: int64

In [5]:
folio_vivienda = folio_vivienda.rename(columns={
    "folio_c": "folio_consecutivo",
    "folio_v": "folio_vivienda"
})

folio_vivienda

,folio_consecutivo,folio_vivienda
0,10321,010321M
1,10322,010321M
2,10800,010800M
3,10801,010800M
4,10842,010842M
...,...,...
50523,321855,321855
50524,321857,321857
50525,321858,321858
50526,321860,321860


In [6]:
folio_vivienda.dtypes

folio_consecutivo     int64
folio_vivienda       object
dtype: object

In [7]:
folio_vivienda["folio_consecutivo"] = folio_vivienda["folio_consecutivo"].astype(str)
folio_vivienda["folio_vivienda"] = folio_vivienda["folio_vivienda"].astype(str)

print(folio_vivienda.dtypes)

folio_consecutivo    object
folio_vivienda       object
dtype: object


In [29]:
folio_vivienda.shape

(50528, 2)

---

### Limpieza Hogar_hogar

In [31]:
Hogar_hogar = pd.read_csv(
    "../data/raw_salud/Hogar_hogar.csv",
    encoding="latin1",  # ← este sí lo lee completo sin error
    na_values=["", " ", "NA"],
    low_memory=False
)


In [32]:
Hogar_hogar.columns = Hogar_hogar.columns.str.replace("ï»¿", "")


In [33]:
# Filtra columnas tipo texto
cols_obj = Hogar_hogar.select_dtypes(include="object").columns

# Buscar valores con caracteres extraños
for col in cols_obj:
    raros = Hogar_hogar[col].dropna().astype(str).str.contains("�", regex=False).sum()
    if raros > 0:
        print(f"{col}: {raros} valores con caracteres extraños")


In [34]:
Hogar_hogar.columns.tolist()

['folio',
 'entidad',
 'munici',
 'locali',
 'deciles',
 'quintiles',
 'h101',
 'h102',
 'h103',
 'h205',
 'h206',
 'h501',
 'h502',
 'h503',
 'h504',
 'h505',
 'h506',
 'h507',
 'h508',
 'h509',
 'h510',
 'h511',
 'h512',
 'h513',
 'h514',
 'h515',
 'h516',
 'h516a',
 'h517',
 'h518',
 'h519',
 'h519esp',
 'h520',
 'h521',
 'h521esp',
 'h522',
 'h522esp',
 'h523',
 'h524a',
 'h524b',
 'h524c',
 'h524d',
 'h524e',
 'h60101',
 'h60102',
 'h60103',
 'h60104',
 'h60105',
 'h60106',
 'h60107',
 'h60108',
 'h60109',
 'h60110',
 'h60111',
 'h60112',
 'h60113',
 'h60114',
 'h60115',
 'h60116',
 'h60117',
 'h60118',
 'h60119',
 'h60120',
 'h60121',
 'h60122',
 'h60123',
 'h60124',
 'h60125',
 'h70101',
 'h70102',
 'h70103',
 'h70104',
 'h70105',
 'h70106',
 'h70107',
 'h70108',
 'h70109',
 'h70110',
 'h70111',
 'h70112',
 'h70113',
 'h70114',
 'h70115',
 'h70116',
 'h70117',
 'h70201',
 'h70202',
 'h70203',
 'h70204',
 'h70205',
 'h70206',
 'h70207',
 'h70301',
 'h70302',
 'h70303',
 'h70304',

In [35]:
columnas_utiles = ['folio', 'entidad', 'munici', 'locali', 'deciles', 'est_urb', 'est_marg', 'pondeh']
hogar_util = Hogar_hogar[columnas_utiles]

hogar_util = hogar_util.rename(columns={
    "folio": "folio_consecutivo"})

Vars:
- folio_consecutivo = identificador unico del hogar
- entidad = clave de la entidad federativa (estado)
- munici = clave del municipio
- locali = clave de la localidad
- deciles = decil de ingreso del hogar
- est_urb = indicador de zona urbana (1 si es urbano, 0 si no)
- est_marg = indicador de marginacion de la zona
- pondeh = ponderador del hogar (factor de expansion muestral)

In [36]:
hogar_util.isna().sum().sort_values(ascending=False)

deciles              2
folio_consecutivo    0
entidad              0
munici               0
locali               0
est_urb              0
est_marg             0
pondeh               0
dtype: int64

In [37]:
hogar_util.shape


(50528, 8)

In [38]:
for col in hogar_util.columns:
    print(f"{col}: {hogar_util[col].nunique()} valores únicos")


folio_consecutivo: 50528 valores únicos
entidad: 32 valores únicos
munici: 177 valores únicos
locali: 323 valores únicos
deciles: 10 valores únicos
est_urb: 3 valores únicos
est_marg: 2 valores únicos
pondeh: 2833 valores únicos


In [39]:
hogar_util['folio_consecutivo'].duplicated().sum()


0

---

### Hogar_integrantes


In [43]:
Hogar_integrantes = pd.read_csv(
    "../data/raw_salud/Hogar_integrantes.csv",
    encoding="latin1",  # ← este sí lo lee completo sin error
    na_values=["", " ", "NA"],
    low_memory=False
)

Hogar_integrantes.columns = Hogar_integrantes.columns.str.replace("ï»¿", "")

In [44]:
Hogar_integrantes

,folio,intp,entidad,munici,locali,edad,meses,dia_nac,mes_nac,anio_nac,...,h401f,h401esp,code_upm,est_dis,est_urb,est_marg,pondei,est_var,afilia_1ra,afilia_tras
0,10001,1,1,1,1,41,10,20,1,1970,...,NaN,NaN,M0101,5,3,1,275.654006,15,1,1
1,10001,2,1,1,1,41,8,1,4,1970,...,NaN,NaN,M0101,5,3,1,264.538414,15,1,1
2,10001,3,1,1,1,19,1,6,11,1992,...,NaN,NaN,M0101,5,3,1,270.517139,15,1,1
3,10001,4,1,1,1,17,9,26,2,1994,...,NaN,NaN,M0101,5,3,1,252.312714,15,1,1
4,10001,5,1,1,1,9,2,6,10,2002,...,NaN,NaN,M0101,5,3,1,297.620993,15,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194918,321861,1,32,57,1,30,6,9,5,1981,...,NaN,NaN,M3248,3,2,1,372.818917,323,0,0
194919,321861,2,32,57,1,28,5,18,6,1983,...,NaN,NaN,M3248,3,2,1,372.796918,323,0,0
194920,321861,3,32,57,1,9,8,8,3,2002,...,NaN,NaN,M3248,3,2,1,407.707324,323,0,0
194921,321861,4,32,57,1,8,9,11,2,2003,...,NaN,NaN,M3248,3,2,1,407.707324,323,0,0


In [46]:
print(Hogar_integrantes.columns.tolist())


['folio', 'intp', 'entidad', 'munici', 'locali', 'edad', 'meses', 'dia_nac', 'mes_nac', 'anio_nac', 'nac', 'sexo', 'h207', 'h207esp', 'h208', 'h208q', 'h209', 'h209q', 'h210a', 'h210b', 'h210esp', 'h211a', 'h211b', 'h211a1', 'h211a2', 'h211c', 'h211b1a', 'h211b1b', 'h212', 'h213a', 'h213b', 'h213esp', 'h214', 'h215', 'h216', 'h217', 'h218a', 'h218b', 'h218a1', 'h219', 'h220', 'h220q', 'h221', 'h222', 'h223', 'h224a', 'h224b', 'h224c', 'h224d', 'h224e', 'h224f', 'h224g', 'h224h', 'h225', 'h226a', 'h226b', 'h227a', 'h227b', 'h227c', 'h227d', 'h227e', 'h227f', 'h301', 'h302', 'h302esp', 'h302a', 'h302b', 'h302c', 'h303', 'h304a', 'h304b', 'h304esp', 'h305a', 'h305b', 'h305c', 'h305esp', 'h305a1', 'h306', 'h306esp', 'h307a', 'h307b', 'h307c', 'h307d', 'h307e', 'h307f', 'h307g', 'h307h', 'h307i', 'h307a1', 'h307a2', 'h307a3', 'h307a4', 'h307a5', 'h307a6', 'h307a7', 'h307a8', 'h308', 'h309a', 'h309b', 'h309c', 'h309esp', 'h310', 'h310a', 'h310aesp', 'h311', 'h312', 'h312esp', 'h313', 'h313es

In [47]:
# Definir columnas relevantes
columnas_utiles_integrantes = [
    'folio',         # folio de vivienda (clave para merge)
    'intp',          # número de persona en el hogar
    'entidad',       # entidad federativa
    'munici',        # municipio
    'locali',        # localidad
    'sexo',          # sexo
    'edad',          # edad en años
    'meses',         # edad en meses
    'dia_nac',       # día de nacimiento
    'mes_nac',       # mes de nacimiento
    'anio_nac',      # año de nacimiento
    'afilia_1ra',    # institución principal de afiliación
    'afilia_tras',   # institución secundaria de afiliación
    'est_urb',       # urbano/rural
    'est_marg',      # nivel de marginación
    'pondei'         # ponderador individual
]

# 3. Filtrar DataFrame con esas columnas
integrantes_util = Hogar_integrantes[columnas_utiles_integrantes].copy()

# 4. Verifica primeras filas y columnas
print(integrantes_util.head())
print(integrantes_util.dtypes)

   folio  intp  entidad  munici  locali  sexo  edad  meses  dia_nac  mes_nac  \
0  10001     1        1       1       1     1    41     10       20        1   
1  10001     2        1       1       1     2    41      8        1        4   
2  10001     3        1       1       1     1    19      1        6       11   
3  10001     4        1       1       1     2    17      9       26        2   
4  10001     5        1       1       1     1     9      2        6       10   

   anio_nac  afilia_1ra  afilia_tras  est_urb  est_marg      pondei  
0      1970           1            1        3         1  275.654006  
1      1970           1            1        3         1  264.538414  
2      1992           1            1        3         1  270.517139  
3      1994           1            1        3         1  252.312714  
4      2002           1            1        3         1  297.620993  
folio            int64
intp             int64
entidad          int64
munici           int64
locali 

Vars:
- folio = identificador unico del hogar
- intp = identificador de la persona dentro del hogar
- entidad = clave de la entidad federativa (estado)
- munici = clave del municipio
- locali = clave de la localidad
- sexo = sexo de la persona (1 = hombre, 2 = mujer)
- edad = edad en años cumplidos
- meses = edad en meses (útil si edad < 1 año)
- dia_nac = dia de nacimiento
- mes_nac = mes de nacimiento
- anio_nac = año de nacimiento
- afilia_1ra = institucion principal a la que esta afiliada la persona (IMSS, ISSSTE, etc.)
- afilia_tras = institucion secundaria de afiliación (si tiene doble cobertura)
- est_urb = indicador de zona urbana (1 si es urbano, 0 si no)
- est_marg = nivel de marginación de la zona
- pondei = ponderador del individuo (factor de expansión muestral)


In [49]:
# nulos por columna
integrantes_util.isnull().sum().sort_values(ascending=False)


folio          0
intp           0
entidad        0
munici         0
locali         0
sexo           0
edad           0
meses          0
dia_nac        0
mes_nac        0
anio_nac       0
afilia_1ra     0
afilia_tras    0
est_urb        0
est_marg       0
pondei         0
dtype: int64

In [51]:
# Dimensiones 
integrantes_util.shape


(194923, 16)

---

### Menores